In [1]:
#import get to call a get request on the site
import requests

#import to manipulate arrays with numpy
import numpy as np

#import to create, clean, and parse data frames with pandas
import pandas as pd

#import to enable datascraping
from bs4 import BeautifulSoup

#import to set up 'sleep' to wait between page loads
import time

# import Mongo so our webscraper dumps its scraped data without losing it
from pymongo import MongoClient
import pymongo

#import to make that html readable
import pprint

#import regular expressions operations
import re

#import to get the universe in balance
import random

#import so we can do some heavy stats work
import scipy as sp
from scipy.stats import binom
import scipy.stats as stats

#import to access certain plotting features
import seaborn as sns

#import because we need its program functions
import math

#import because we need to plot and make it pretty
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
%matplotlib inline
plt.style.use('ggplot')

In [2]:
# This will collect the links for all cities on craigslist. We will have to sparse through
# these and cut out the non-US cities.
def city_link_collector():
    # this is the craigslist page with every city
    main_page = requests.get('https://www.craigslist.org/about/sites')
    soup = BeautifulSoup(main_page.text, 'html.parser')
    
    all_list = []
    uscity_list = []
    city_list = []
    
    for i in range(4):
        for box in soup.find_all('div', class_='box box_{}'.format(i+1)):
            all_list.append(box.find_all('a'))
    for _ in all_list[:20:7]:
        for __ in _:
            uscity_list.append(__)
    for ___ in all_list[20][0:94]:
        uscity_list.append(___)
    
    for idx, city in enumerate(uscity_list):
        city_list.append(str(uscity_list[idx]).split('''"''')[1])
    
    '''These listings are all a subset of Miami and break the scraper if entered in this format'''
    city_list.remove('http://miami.craigslist.org/brw/')
    city_list.remove('http://miami.craigslist.org/mdc/')
    city_list.remove('http://miami.craigslist.org/pbc/')
    '''Here is the Miami list to capture the dropped entries and will not break the scraper'''
    city_list.append('http://miami.craigslist.org/')
    return city_list
    #                     #posting date
    #                     #grab the datetime element 0 for date and 1 for time
    #                     post_datetime = box.find('time', class_= 'result-date')['datetime']
    #                     post_timing.append(post_datetime)

            

In [3]:
city_list = city_link_collector()
len(city_list)

415

In [4]:
def craigslist_motorcycle_scraper(city_list):
    client = MongoClient('localhost', 27017)
    db = client['craigslist_motorcycles']
    post_html = db['motorcycle_posts']
    for city in city_list: 

        #get the first page of the Austin motorcycle prices
        city_response = requests.get('{}search/mca?s=0&bundleDuplicates=1'.format(city))
        #parse through it and make it readable
        html_soup = BeautifulSoup(city_response.text, 'html.parser')
        #find the total number of posts to find the limit for each page
        results_num = html_soup.find('div', class_= 'search-legend')
        #pulled the total count of posts as the upper bound of the pages array
        results_total = int(results_num.find('span', class_='totalcount').text) 
        #each page has 119 posts so each new page is defined as follows: s=120, s=240, s=360, and so on. So we need to step in size 120 in the np.arange function
        pages = np.arange(0, results_total+1, 120)
        print("{} Posts = {}".format(city, results_total))
        print("{} Pages = {}".format(city.title(), len(pages)))

        iterations = 0

        for page in pages:         
            
            #get request      
            post_response = requests.get("{}search/mca?".format(city) 
                           + "s=" #the parameter for defining the page number 
                           + str(page) #the page number in the pages array from earlier
                           + "&bundleDuplicates=1")

            time.sleep(random.randint(2,3))

            #throw warning for status codes that are not 200
            if post_response.status_code != 200:
                warn('Request: {}; Status code: {}'.format(requests, post_response.status_code))

            #define the html text
            post_soup = BeautifulSoup(post_response.text, 'html.parser')

            count = 0
            for post in post_soup.find_all('a', class_ = 'result-title hdrlnk'):
                link = post['href']
                count += 1
                
#               checks to see if post html is loaded in our database and scrapes html if not
                if not (post_html.find_one({'_id': link}, {'html': requests.get(link).text})):
                    print(str(link))
                    sub_post = requests.get(link)
                    post_html.insert_one({'_id': link, 'html': sub_post.text})
                    if sub_post.status_code != 200:
                        warn('Request: {}; Status code: {}'.format(requests, post_response.status_code))
                    time.sleep(random.randint(2,3)) #sleep timer to avoid being banned      
                else:
                    print('Passing ' + str(link))
                    pass
                         
                if count == results_total:
                    break

            iterations += 1
            print("{} Page ".format(city.title()) + str(iterations) + " of {} pages".format(len(pages)) + " scraped successfully!")

            

        print("\n")
        print("{} complete!".format(city.title()))
        print('~' + str(len(pages)*120) + " rows collected.")
        print("\n")

In [5]:
client = MongoClient('localhost', 27017)
db = client['craigslist_motorcycles']
post_html = db['motorcycle_posts']

In [ ]:
craigslist_motorcycle_scraper(city_list[15:])

https://phoenix.craigslist.org/ Posts = 2366
Https://Phoenix.Craigslist.Org/ Pages = 20
https://phoenix.craigslist.org/nph/mcy/d/cave-creek-yamaha-vmax-1200/7122469567.html
https://phoenix.craigslist.org/wvl/mcd/d/phoenix-2015-harley-davidson-fltrx-road/7122466722.html
https://phoenix.craigslist.org/evl/mcd/d/mesa-2019-suzuki-boulevard-c90-o-s/7122466658.html
https://phoenix.craigslist.org/evl/mcy/d/chandler-2005-yamaha-r6/7122450357.html
https://phoenix.craigslist.org/evl/mcy/d/florence-2008-ktm-530-exc/7122422454.html
https://phoenix.craigslist.org/nph/mcd/d/cornville-2020-kawasaki-klx-110l/7122422397.html
https://phoenix.craigslist.org/nph/mcy/d/scottsdale-2016-crf450r/7122420125.html
https://phoenix.craigslist.org/nph/mcy/d/phoenix-2019-beta-xtrainer/7122419880.html
https://phoenix.craigslist.org/evl/mcy/d/mesa-2007-ultra-ground-pounder/7122408903.html
Passing https://phoenix.craigslist.org/nph/mcd/d/cornville-2018-kawasaki-versys-1000-lt/7122406800.html
Passing https://phoenix.cra

Passing https://phoenix.craigslist.org/wvl/mcd/d/peoria-2017-suzuki-boulevard-s40/7122155765.html
Passing https://phoenix.craigslist.org/wvl/mcd/d/peoria-2019-harley-davidson-flhtcu/7122155772.html
Passing https://phoenix.craigslist.org/nph/mcd/d/cornville-2019-kawasaki-kx-65-skua095926/7122149110.html
Passing https://phoenix.craigslist.org/nph/mcy/d/scottsdale-harley-cvo-street-glide/7122144252.html
Passing https://phoenix.craigslist.org/cph/mcy/d/casa-grande-chopper-rigid-with-harley/7122098389.html
Passing https://phoenix.craigslist.org/wvl/mcy/d/glendale-zero-model-72/7122067619.html
Passing https://phoenix.craigslist.org/wvl/mcy/d/goodyear-honda-goldwing-gl-1800/7122067441.html
Passing https://phoenix.craigslist.org/nph/mcy/d/cave-creek-2017-ktm-690-enduro-loaded/7122049602.html
Passing https://phoenix.craigslist.org/evl/mcy/d/tempe-500af/7122048061.html
Passing https://phoenix.craigslist.org/evl/mcy/d/rio-verde-suzuki-burgman-400-abs/7122018072.html
Passing https://phoenix.craigs

Passing https://phoenix.craigslist.org/wvl/mcy/d/peoria-harley-davidson-road-king-2001/7121557590.html
Passing https://phoenix.craigslist.org/evl/mcy/d/scottsdale-2014-cbr500r/7121555749.html
Passing https://phoenix.craigslist.org/nph/mcy/d/phoenix-2013-v7-racer/7121551582.html
Passing https://phoenix.craigslist.org/evl/mcd/d/tempe-2004-harley-davidson-xl1200-we/7121549650.html
Passing https://phoenix.craigslist.org/wvl/mcd/d/2015-yamaha-yzf-r1-we-want-your-trade/7121548887.html
Passing https://phoenix.craigslist.org/evl/mcd/d/tempe-2015-yamaha-yzf-r1-we-buy-bikes/7121486857.html
Passing https://phoenix.craigslist.org/wvl/mcd/d/2018-yamaha-mt-07-we-want-your-trade/7121533912.html
Passing https://phoenix.craigslist.org/evl/mcd/d/tempe-2018-suzuki-hayabusa-we-buy-bikes/7121530153.html
Passing https://phoenix.craigslist.org/evl/mcy/d/mesa-husqvarna-fx/7121528276.html
Passing https://phoenix.craigslist.org/evl/mcd/d/mesa-2017-indian-chieftain-black-pearl/7121524732.html
Passing https://pho

Passing https://phoenix.craigslist.org/nph/mcd/d/phoenix-2014-harley-davidson-heritage/7121272271.html
Https://Phoenix.Craigslist.Org/ Page 2 of 20 pages scraped successfully!
Passing https://phoenix.craigslist.org/nph/mcd/d/phoenix-2014-harley-davidson-softail/7121272637.html
Passing https://phoenix.craigslist.org/nph/mcd/d/phoenix-2014-harley-davidson-heritage/7121272271.html
Passing https://phoenix.craigslist.org/nph/mcd/d/phoenix-2016-harley-davidson-heritage/7121271928.html
Passing https://phoenix.craigslist.org/nph/mcd/d/phoenix-2016-harley-davidson-ultra/7121271482.html
Passing https://phoenix.craigslist.org/nph/mcd/d/phoenix-2015-harley-davidson-road-glide/7121270913.html
Passing https://phoenix.craigslist.org/nph/mcy/d/phoenix-2018-ducati-panigale-corse-959/7121259159.html
Passing https://phoenix.craigslist.org/evl/mcd/d/mesa-2019-harley-davidson-softail-fat/7121251678.html
Passing https://phoenix.craigslist.org/evl/mcy/d/gilbert-2005-street-glide-flhti/7121249286.html
Passing

Passing https://phoenix.craigslist.org/evl/mcd/d/tempe-2007-honda-vtx1300-we-buy-bikes/7121021082.html
Passing https://phoenix.craigslist.org/evl/mcd/d/tempe-2007-honda-vtx1300-we-buy-bikes/7119765110.html
Passing https://phoenix.craigslist.org/evl/mcy/d/scottsdale-2014-moto-guzzi/7121019245.html
Passing https://phoenix.craigslist.org/evl/mcd/d/tempe-2007-harley-davidson-fat-boy-we/7121013953.html
Passing https://phoenix.craigslist.org/evl/mcd/d/tempe-2014-honda-cb1100e/7121010715.html
Passing https://phoenix.craigslist.org/wvl/mcd/d/2016-harley-davidson-xg750-street-rod/7121009953.html
Passing https://phoenix.craigslist.org/evl/mcd/d/tempe-2016-triumph-street-triple/7121009631.html
Passing https://phoenix.craigslist.org/evl/mcd/d/tempe-2007-yamaha-star/7121007947.html
Passing https://phoenix.craigslist.org/evl/mcd/d/tempe-2017-honda-cbr1000rr-we-buy-bikes/7121006041.html
Passing https://phoenix.craigslist.org/nph/mcy/d/phoenix-1982-honda-ct70/7121000161.html
Passing https://phoenix.cr

...

Passing https://phoenix.craigslist.org/cph/mcd/d/tempe-new-2020-harley-davidson-iron-883/7115187680.html
Passing https://phoenix.craigslist.org/cph/mcd/d/tempe-new-2020-harley-davidson-tri/7115187520.html
Passing https://phoenix.craigslist.org/cph/mcd/d/tempe-new-2020-harley-davidson-street/7115187311.html
Passing https://phoenix.craigslist.org/wvl/mcd/d/peoria-2018-honda-gold-wing-candy/7115186878.html
Passing https://phoenix.craigslist.org/wvl/mcd/d/peoria-2019-honda-nc750x/7115186842.html
Passing https://phoenix.craigslist.org/nph/mcy/d/phoenix-2007-polaris-outlaw-90cc/7115139793.html
Passing https://phoenix.craigslist.org/nph/mcy/d/phoenix-2013-harley-davidson-sportster/7115131454.html
Passing https://phoenix.craigslist.org/evl/mcy/d/phoenix-pristine-harley-davidson-1200/7115109277.html
Passing https://phoenix.craigslist.org/cph/mcy/d/tempe-triumph-tiger-955i/7115060342.html
Passing https://phoenix.craigslist.org/evl/mcy/d/payson-2016-hd-street-glide-special/7114989284.html
Passing

Passing https://phoenix.craigslist.org/evl/mcy/d/queen-creek-aprilia-550-rxv-2007/7114250287.html
Passing https://phoenix.craigslist.org/evl/mcy/d/scottsdale-2006-kawasaki-concours-zg1000/7114239725.html
Passing https://phoenix.craigslist.org/evl/mcy/d/chandler-harley-davidson-sportster/7114238642.html
Passing https://phoenix.craigslist.org/cph/mcd/d/valyermo-2020-csc-rx3-250cc-adventure/7114207813.html
Passing https://phoenix.craigslist.org/evl/mcd/d/mesa-2019-harley-davidson-road-king-flhr/7114206505.html
Passing https://phoenix.craigslist.org/cph/mcy/d/tucson-2010-yamaha-vstar-650/7114194361.html
Passing https://phoenix.craigslist.org/nph/mcy/d/rimrock-1950-harley-davidson-trike/7114180007.html
Passing https://phoenix.craigslist.org/cph/mcd/d/tempe-used-2019-harley-davidson-electra/7114179079.html
Passing https://phoenix.craigslist.org/wvl/mcd/d/avondale-used-2019-harley-davidson-fxlr/7114176566.html
Passing https://phoenix.craigslist.org/wvl/mcd/d/avondale-used-2018-suzuki-gsx-r100

Passing https://phoenix.craigslist.org/wvl/mcd/d/peoria-used-2019-harley-davidson-road/7113827700.html
https://phoenix.craigslist.org/wvl/mcy/d/sun-city-west-motorcycle-goldwing/7113813976.html
https://phoenix.craigslist.org/evl/mcd/d/mesa-2013-harley-davidson-flhtcu-ultra/7113807620.html
https://phoenix.craigslist.org/cph/mcy/d/phoenix-1971-hodaka-ace/7113798245.html
https://phoenix.craigslist.org/nph/mcd/d/phoenix-2019-yamaha-yz450f/7113786444.html
https://phoenix.craigslist.org/evl/mcd/d/mesa-2017-harley-davidson-flhtcutg/7113779856.html
https://phoenix.craigslist.org/evl/mcd/d/mesa-2014-harley-davidson-flhtkse-cvo/7113754210.html
https://phoenix.craigslist.org/wvl/mcy/d/peoria-2006-ktm-525sx/7113744816.html
https://phoenix.craigslist.org/evl/mcy/d/gilbert-harley-davidson-road-glide/7113719585.html
https://phoenix.craigslist.org/nph/mcy/d/cornville-2015-triumph-bonneville-or/7113708889.html
https://phoenix.craigslist.org/nph/mcy/d/phoenix-2018-ducati-panigale-corse-959/7113698914.ht

https://phoenix.craigslist.org/evl/mcd/d/chandler-used-2019-harley-davidson/7113118331.html
https://phoenix.craigslist.org/evl/mcd/d/chandler-used-2013-honda-gold-wing-f6b/7113118312.html
https://phoenix.craigslist.org/evl/mcd/d/chandler-used-2016-harley-davidson-flss/7113118313.html
https://phoenix.craigslist.org/evl/mcd/d/chandler-used-2019-harley-davidson-fxlr/7113118307.html
https://phoenix.craigslist.org/evl/mcd/d/chandler-used-2019-harley-davidson-flhc/7113118300.html
https://phoenix.craigslist.org/evl/mcd/d/chandler-used-2018-bmw-1200-rt-blue/7113118302.html
https://phoenix.craigslist.org/evl/mcd/d/chandler-used-2018-ducati-monster-821/7113118286.html
https://phoenix.craigslist.org/evl/mcd/d/chandler-used-2017-honda-rebel-500-in/7113118258.html
https://phoenix.craigslist.org/cph/mcd/d/tempe-new-2020-harley-davidson-cvo/7113115930.html
https://phoenix.craigslist.org/cph/mcd/d/tempe-new-2020-harley-davidson-tri/7113115194.html
https://phoenix.craigslist.org/wvl/mcd/d/peoria-used-2

https://phoenix.craigslist.org/evl/mcd/d/chandler-used-2018-ktm-1290-super-duke/7112609071.html
https://phoenix.craigslist.org/evl/mcd/d/chandler-used-2017-harley-davidson/7112609058.html
https://phoenix.craigslist.org/evl/mcd/d/chandler-used-2018-bmw-310-gs-racing/7112609016.html
https://phoenix.craigslist.org/evl/mcd/d/chandler-used-2019-honda-crf450l-in-red/7112609030.html
https://phoenix.craigslist.org/evl/mcd/d/chandler-used-2019-harley-davidson-flhc/7112609014.html
https://phoenix.craigslist.org/nph/mcy/d/scottsdale-2015-harley-davidson-fat-boy/7112603660.html
https://phoenix.craigslist.org/cph/mcd/d/tempe-new-2020-harley-davidson-low/7112588762.html
https://phoenix.craigslist.org/cph/mcd/d/tempe-new-2020-harley-davidson-heritage/7112588679.html
https://phoenix.craigslist.org/wvl/mcd/d/peoria-used-2019-harley-davidson/7112588584.html
https://phoenix.craigslist.org/wvl/mcd/d/peoria-used-2019-harley-davidson/7112588492.html
https://phoenix.craigslist.org/cph/mcd/d/tempe-new-2020-ha